In [ ]:
import pandas as pd
import ast
import json
import re

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162670 sha256=c987e4c10e18aa508a3804965cac23a255256291a2254f524538477a8087eae7
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy

# Datasets

In [71]:
df_restaurants = pd.read_csv('/content/drive/MyDrive/MaestriaDataMining/SistemaDeRecomendación/Scraping/restaurants_cafe.csv', index_col='id')

In [ ]:
df_users = pd.read_csv('/content/drive/MyDrive/MaestriaDataMining/SistemaDeRecomendación/Scraping/usuarios_cafe.csv', index_col='user_id')

In [ ]:
df_interactions = pd.read_csv('/content/drive/MyDrive/MaestriaDataMining/SistemaDeRecomendación/Scraping/interacciones_cafe.csv')

#Limpieza de datos

## Restaurants

### Info

In [72]:
df_restaurants.shape

(572, 24)

In [73]:
df_restaurants.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 572 entries, 10048394 to 25320045
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              572 non-null    object 
 1   num_reviews       572 non-null    int64  
 2   latitude          571 non-null    float64
 3   longitude         571 non-null    float64
 4   timezone          572 non-null    object 
 5   location          572 non-null    object 
 6   raw_ranking       498 non-null    float64
 7   ranking_position  498 non-null    float64
 8   ranking_caba      498 non-null    float64
 9   rating            498 non-null    float64
 10  price             20 non-null     object 
 11  price_level       352 non-null    object 
 12  phone             409 non-null    object 
 13  website           471 non-null    object 
 14  email             197 non-null    object 
 15  address           572 non-null    object 
 16  postalcode        481 non-null  

###Cambiar index

Para poder unificar el nombre del campo que contiene el ID único de restaurants para tener el mismo nombre del campo que la tabla o dataset de interacciones.


In [74]:
df_restaurants = df_restaurants.rename_axis('restaurant_id').reset_index()

In [75]:
df_restaurants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572 entries, 0 to 571
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   restaurant_id     572 non-null    int64  
 1   name              572 non-null    object 
 2   num_reviews       572 non-null    int64  
 3   latitude          571 non-null    float64
 4   longitude         571 non-null    float64
 5   timezone          572 non-null    object 
 6   location          572 non-null    object 
 7   raw_ranking       498 non-null    float64
 8   ranking_position  498 non-null    float64
 9   ranking_caba      498 non-null    float64
 10  rating            498 non-null    float64
 11  price             20 non-null     object 
 12  price_level       352 non-null    object 
 13  phone             409 non-null    object 
 14  website           471 non-null    object 
 15  email             197 non-null    object 
 16  address           572 non-null    object 
 1

In [76]:
df_restaurants.set_index('restaurant_id', inplace=True)

In [77]:
df_restaurants.head()

,name,num_reviews,latitude,longitude,timezone,location,raw_ranking,ranking_position,ranking_caba,rating,...,email,address,postalcode,image_url,type,url,special_diets,califications,traveler_rating,display_hours
restaurant_id,,,,,,,,,,,,,,,,,,,,,
10048394,Vive Cafe,357,-34.581840,-58.436085,America/Argentina/Buenos_Aires,"Buenos Aires, Distrito Capital Federal",4.716580,21.0,1.0,5.0,...,vivecafe@outlook.com,"Costa Rica 5722, Buenos Aires 1414 Argentina",1414,https://media-cdn.tripadvisor.com/media/photo-...,"['$', 'Café', 'Colombiana', 'Apto para vegetar...",https://www.tripadvisor.com.ar/Restaurant_Revi...,"Apto para vegetarianos, Opciones veganas, Opci...","{'Comida': 4.5, 'Servicio': 4.5, 'Calidad/prec...","{'Excelente': '163', 'Muy bueno': '38', 'Regul...","[{'days': 'lun - vie', 'times': ['08:30 - 20:0..."
2424768,Full City Coffee House,460,-34.588444,-58.431625,America/Argentina/Buenos_Aires,"Buenos Aires, Distrito Capital Federal",4.438332,56.0,2.0,4.5,...,fullcityba@gmail.com,"Thames 1535 Palermo Soho, Buenos Aires Argentina",NaN,NaN,"['$$ - $$$', 'Café', 'Apto para vegetarianos',...",https://www.tripadvisor.com.ar/Restaurant_Revi...,"Apto para vegetarianos, Opciones veganas","{'Comida': 4.5, 'Servicio': 4.5, 'Calidad/prec...","{'Excelente': '233', 'Muy bueno': '72', 'Regul...","[{'days': 'dom', 'times': ['11:00 - 18:00']}, ..."
1902671,Havanna,1805,-34.601048,-58.375390,America/Argentina/Buenos_Aires,"Buenos Aires, Distrito Capital Federal",4.175300,125.0,8.0,4.5,...,customerserv@havanna.com.ar,"Calle Florida 525, Buenos Aires C1005AAK Argen...",C1005AAK,https://media-cdn.tripadvisor.com/media/photo-...,"['$$ - $$$', 'Postres', 'Café', 'Fuente de soda']",https://www.tripadvisor.com.ar/Restaurant_Revi...,NaN,"{'Comida': 4.0, 'Servicio': 4.0, 'Calidad/prec...","{'Excelente': '422', 'Muy bueno': '496', 'Regu...",NaN
3676783,El Gato Negro,858,-34.604305,-58.390320,America/Argentina/Buenos_Aires,"Buenos Aires, Distrito Capital Federal",4.092933,157.0,4.0,4.0,...,NaN,"Av. Corrientes 1669, entre Montevideo y Rodríg...",C1042AAC,https://media-cdn.tripadvisor.com/media/photo-...,"['$$ - $$$', 'Café', 'Internacional', 'Apto pa...",https://www.tripadvisor.com.ar/Restaurant_Revi...,"Apto para vegetarianos, Opciones sin gluten","{'Comida': 4.0, 'Servicio': 4.0, 'Calidad/prec...","{'Excelente': '263', 'Muy bueno': '261', 'Regu...","[{'days': 'dom', 'times': ['15:00 - 23:00']}, ..."
6563785,Petit Colon,760,-34.602490,-58.383896,America/Argentina/Buenos_Aires,"Buenos Aires, Distrito Capital Federal",4.044234,183.0,5.0,4.0,...,info@elblogdelpetitcolon.com.ar,"Libertad 505, esquina Lavalle- Zona Tribunales...",NaN,https://media-cdn.tripadvisor.com/media/photo-...,"['$$ - $$$', 'Café', 'Argentina', 'Apto para v...",https://www.tripadvisor.com.ar/Restaurant_Revi...,Apto para vegetarianos,"{'Comida': 4.0, 'Servicio': 4.0, 'Calidad/prec...","{'Excelente': '213', 'Muy bueno': '232', 'Regu...","[{'days': 'dom - sáb', 'times': ['08:00 - 01:0..."


###Eliminar duplicados

In [78]:
df_restaurants = df_restaurants[~df_restaurants.index.duplicated(keep='first')]

In [79]:
df_restaurants.shape

(530, 24)

In [80]:
df_restaurants.head(5)

,name,num_reviews,latitude,longitude,timezone,location,raw_ranking,ranking_position,ranking_caba,rating,...,email,address,postalcode,image_url,type,url,special_diets,califications,traveler_rating,display_hours
restaurant_id,,,,,,,,,,,,,,,,,,,,,
10048394,Vive Cafe,357,-34.581840,-58.436085,America/Argentina/Buenos_Aires,"Buenos Aires, Distrito Capital Federal",4.716580,21.0,1.0,5.0,...,vivecafe@outlook.com,"Costa Rica 5722, Buenos Aires 1414 Argentina",1414,https://media-cdn.tripadvisor.com/media/photo-...,"['$', 'Café', 'Colombiana', 'Apto para vegetar...",https://www.tripadvisor.com.ar/Restaurant_Revi...,"Apto para vegetarianos, Opciones veganas, Opci...","{'Comida': 4.5, 'Servicio': 4.5, 'Calidad/prec...","{'Excelente': '163', 'Muy bueno': '38', 'Regul...","[{'days': 'lun - vie', 'times': ['08:30 - 20:0..."
2424768,Full City Coffee House,460,-34.588444,-58.431625,America/Argentina/Buenos_Aires,"Buenos Aires, Distrito Capital Federal",4.438332,56.0,2.0,4.5,...,fullcityba@gmail.com,"Thames 1535 Palermo Soho, Buenos Aires Argentina",NaN,NaN,"['$$ - $$$', 'Café', 'Apto para vegetarianos',...",https://www.tripadvisor.com.ar/Restaurant_Revi...,"Apto para vegetarianos, Opciones veganas","{'Comida': 4.5, 'Servicio': 4.5, 'Calidad/prec...","{'Excelente': '233', 'Muy bueno': '72', 'Regul...","[{'days': 'dom', 'times': ['11:00 - 18:00']}, ..."
1902671,Havanna,1805,-34.601048,-58.375390,America/Argentina/Buenos_Aires,"Buenos Aires, Distrito Capital Federal",4.175300,125.0,8.0,4.5,...,customerserv@havanna.com.ar,"Calle Florida 525, Buenos Aires C1005AAK Argen...",C1005AAK,https://media-cdn.tripadvisor.com/media/photo-...,"['$$ - $$$', 'Postres', 'Café', 'Fuente de soda']",https://www.tripadvisor.com.ar/Restaurant_Revi...,NaN,"{'Comida': 4.0, 'Servicio': 4.0, 'Calidad/prec...","{'Excelente': '422', 'Muy bueno': '496', 'Regu...",NaN
3676783,El Gato Negro,858,-34.604305,-58.390320,America/Argentina/Buenos_Aires,"Buenos Aires, Distrito Capital Federal",4.092933,157.0,4.0,4.0,...,NaN,"Av. Corrientes 1669, entre Montevideo y Rodríg...",C1042AAC,https://media-cdn.tripadvisor.com/media/photo-...,"['$$ - $$$', 'Café', 'Internacional', 'Apto pa...",https://www.tripadvisor.com.ar/Restaurant_Revi...,"Apto para vegetarianos, Opciones sin gluten","{'Comida': 4.0, 'Servicio': 4.0, 'Calidad/prec...","{'Excelente': '263', 'Muy bueno': '261', 'Regu...","[{'days': 'dom', 'times': ['15:00 - 23:00']}, ..."
6563785,Petit Colon,760,-34.602490,-58.383896,America/Argentina/Buenos_Aires,"Buenos Aires, Distrito Capital Federal",4.044234,183.0,5.0,4.0,...,info@elblogdelpetitcolon.com.ar,"Libertad 505, esquina Lavalle- Zona Tribunales...",NaN,https://media-cdn.tripadvisor.com/media/photo-...,"['$$ - $$$', 'Café', 'Argentina', 'Apto para v...",https://www.tripadvisor.com.ar/Restaurant_Revi...,Apto para vegetarianos,"{'Comida': 4.0, 'Servicio': 4.0, 'Calidad/prec...","{'Excelente': '213', 'Muy bueno': '232', 'Regu...","[{'days': 'dom - sáb', 'times': ['08:00 - 01:0..."


###Separar texto en columnas

Del scraping hay datos que se guardaron como Array o JSON, se separan estos datos en columnas para el análisis

In [81]:
df_restaurants['traveler_rating']

restaurant_id
10048394    {'Excelente': '163', 'Muy bueno': '38', 'Regul...
2424768     {'Excelente': '233', 'Muy bueno': '72', 'Regul...
1902671     {'Excelente': '422', 'Muy bueno': '496', 'Regu...
3676783     {'Excelente': '263', 'Muy bueno': '261', 'Regu...
6563785     {'Excelente': '213', 'Muy bueno': '232', 'Regu...
                                  ...                        
26768472                                                   {}
25762339                                                   {}
25330579                                                   {}
25944091                                                   {}
25320045                                                   {}
Name: traveler_rating, Length: 530, dtype: object

In [82]:
df_restaurants['traveler_rating'] = df_restaurants['traveler_rating'].apply(ast.literal_eval)
df_restaurants[['traveler_rating_excelente', 'traveler_rating_muy_bueno', 'traveler_rating_regular', 'traveler_rating_malo', 'traveler_rating_horrible']] = pd.DataFrame(df_restaurants['traveler_rating'].tolist(), index=df_restaurants.index)

# Eliminar la columna original 'traveler_rating'
df_restaurants = df_restaurants.drop('traveler_rating', axis=1)


In [83]:
df_restaurants['califications'] = df_restaurants['califications'].apply(ast.literal_eval)
df_restaurants[['califications_comida', 'califications_servicio', 'califications_calidad_precio', 'califications_ambiente']] = pd.DataFrame(df_restaurants['califications'].tolist(), index=df_restaurants.index)

# Eliminar la columna original 'califications'
df_restaurants = df_restaurants.drop('califications', axis=1)

In [84]:
df_restaurants.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 530 entries, 10048394 to 25320045
Data columns (total 31 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   name                          530 non-null    object 
 1   num_reviews                   530 non-null    int64  
 2   latitude                      529 non-null    float64
 3   longitude                     529 non-null    float64
 4   timezone                      530 non-null    object 
 5   location                      530 non-null    object 
 6   raw_ranking                   467 non-null    float64
 7   ranking_position              467 non-null    float64
 8   ranking_caba                  467 non-null    float64
 9   rating                        467 non-null    float64
 10  price                         19 non-null     object 
 11  price_level                   332 non-null    object 
 12  phone                         379 non-null    object

In [85]:
df_restaurants['califications_ambiente'].value_counts

<bound method IndexOpsMixin.value_counts of restaurant_id
10048394    3.5
2424768     4.5
1902671     4.0
3676783     4.5
6563785     5.0
           ... 
26768472    NaN
25762339    NaN
25330579    NaN
25944091    NaN
25320045    NaN
Name: califications_ambiente, Length: 530, dtype: float64>

La columna traveler rating, tenía datos guardados como decimales cuando el valor es un número entero, se ajusta el tipo de dato para evitar confusiones futuras.

In [86]:
columnas_a_convertir = ['traveler_rating_excelente', 'traveler_rating_muy_bueno', 'traveler_rating_regular', 'traveler_rating_malo', 'traveler_rating_horrible']

for columna in columnas_a_convertir:
    df_restaurants[columna] = df_restaurants[columna].fillna(0).astype(float).astype(int)

### Obtener tags de *special_diets*

En el campo *special_diets* se tiene información de si el restaurant-café tiene menú vegano, vegetariano y sin gluten

In [87]:
df_restaurants['special_diets'].unique()

array(['Apto para vegetarianos, Opciones veganas, Opciones sin gluten',
       'Apto para vegetarianos, Opciones veganas', nan,
       'Apto para vegetarianos, Opciones sin gluten',
       'Apto para vegetarianos', 'Opciones veganas',
       'Opciones sin gluten, Apto para vegetarianos'], dtype=object)

In [88]:
df_restaurants['vegano'] = df_restaurants['special_diets'].apply(lambda x: 1 if pd.notnull(x) and 'vegano' in str(x).lower() else 0)
df_restaurants['sin_gluten'] = df_restaurants['special_diets'].apply(lambda x: 1 if pd.notnull(x) and 'sin gluten' in str(x).lower() else 0)
df_restaurants['vegetariano'] = df_restaurants['special_diets'].apply(lambda x: 1 if pd.notnull(x) and 'vegetariano' in str(x).lower() else 0)

In [89]:
df_restaurants['special_diets'].head(10)

restaurant_id
10048394    Apto para vegetarianos, Opciones veganas, Opci...
2424768              Apto para vegetarianos, Opciones veganas
1902671                                                   NaN
3676783           Apto para vegetarianos, Opciones sin gluten
6563785                                Apto para vegetarianos
3708642              Apto para vegetarianos, Opciones veganas
806778                                                    NaN
2311012                                                   NaN
946017            Apto para vegetarianos, Opciones sin gluten
12607745                                                  NaN
Name: special_diets, dtype: object

In [90]:
df_restaurants.head(10)

,name,num_reviews,latitude,longitude,timezone,location,raw_ranking,ranking_position,ranking_caba,rating,...,traveler_rating_regular,traveler_rating_malo,traveler_rating_horrible,califications_comida,califications_servicio,califications_calidad_precio,califications_ambiente,vegano,sin_gluten,vegetariano
restaurant_id,,,,,,,,,,,,,,,,,,,,,
10048394,Vive Cafe,357,-34.581840,-58.436085,America/Argentina/Buenos_Aires,"Buenos Aires, Distrito Capital Federal",4.716580,21.0,1.0,5.0,...,8,2,2,4.5,4.5,4.5,3.5,0,1,1
2424768,Full City Coffee House,460,-34.588444,-58.431625,America/Argentina/Buenos_Aires,"Buenos Aires, Distrito Capital Federal",4.438332,56.0,2.0,4.5,...,25,8,3,4.5,4.5,4.5,4.5,0,0,1
1902671,Havanna,1805,-34.601048,-58.375390,America/Argentina/Buenos_Aires,"Buenos Aires, Distrito Capital Federal",4.175300,125.0,8.0,4.5,...,176,17,9,4.0,4.0,3.5,4.0,0,0,0
3676783,El Gato Negro,858,-34.604305,-58.390320,America/Argentina/Buenos_Aires,"Buenos Aires, Distrito Capital Federal",4.092933,157.0,4.0,4.0,...,67,24,12,4.0,4.0,4.0,4.5,0,1,1
6563785,Petit Colon,760,-34.602490,-58.383896,America/Argentina/Buenos_Aires,"Buenos Aires, Distrito Capital Federal",4.044234,183.0,5.0,4.0,...,60,16,13,4.0,4.0,3.5,5.0,0,0,1
3708642,Cocu Boulangerie,777,-34.591220,-58.428060,America/Argentina/Buenos_Aires,"Buenos Aires, Distrito Capital Federal",4.000464,204.0,3.0,4.0,...,65,33,20,4.5,4.0,4.0,4.0,0,0,1
806778,Café Tortoni,20040,-34.608727,-58.378170,America/Argentina/Buenos_Aires,"Buenos Aires, Distrito Capital Federal",3.983110,220.0,13.0,4.0,...,835,214,105,4.0,4.0,3.5,4.5,0,0,0
2311012,Havanna Cafe,485,-34.589558,-58.409466,America/Argentina/Buenos_Aires,"Buenos Aires, Distrito Capital Federal",3.975816,227.0,8.0,4.0,...,38,11,11,4.0,4.0,3.5,4.0,0,0,0
946017,Las Violetas,1903,-34.611150,-58.420788,America/Argentina/Buenos_Aires,"Buenos Aires, Distrito Capital Federal",3.971121,229.0,15.0,4.0,...,194,75,42,4.0,4.0,3.5,4.5,0,1,1


In [91]:
df_restaurants = df_restaurants.drop('special_diets', axis=1)

### Análisis de campo *type*

Se evalúa si el campo type es relevante y si tiene relevancia para el análisis y para caracterizar un restaurant.

In [92]:
df_restaurants['type'].unique()

array(["['$', 'Café', 'Colombiana', 'Apto para vegetarianos']",
       "['$$ - $$$', 'Café', 'Apto para vegetarianos', 'Opciones veganas']",
       "['$$ - $$$', 'Postres', 'Café', 'Fuente de soda']",
       "['$$ - $$$', 'Café', 'Internacional', 'Apto para vegetarianos']",
       "['$$ - $$$', 'Café', 'Argentina', 'Apto para vegetarianos']",
       "['$$ - $$$', 'Panaderías y confiterías\\xa0', 'Francesa', 'Café']",
       "['$$ - $$$', 'Postres', 'Apto para vegetarianos']",
       "['$$ - $$$', 'Café']",
       "['$$ - $$$', 'Postres', 'Café', 'Argentina']",
       "['$', 'Café', 'Apto para vegetarianos']",
       "['$$ - $$$', 'Panaderías y confiterías\\xa0', 'Cajún y criolla', 'Café']",
       "['$$ - $$$', 'Café', 'Apto para vegetarianos']",
       "['$$ - $$$', 'Postres', 'Café', 'Internacional']",
       "['$$ - $$$', 'Cervecería', 'Café', 'Comida rápida']",
       "['$$ - $$$', 'Tentempiés', 'Estadounidense', 'Café']",
       "['$$ - $$$', 'Café', 'Sudamericana', 'Apto para veg

In [93]:
df_restaurants['combinacion_tipos_comida'] = df_restaurants['type'].apply(lambda x: ', '.join(eval(x)) if pd.notnull(x) else None)
frecuencia_combinaciones = df_restaurants['combinacion_tipos_comida'].value_counts()
print(frecuencia_combinaciones)

Café                                                   81
$$ - $$$, Café                                         64
$$ - $$$, Café, Argentina, Sudamericana                45
                                                       30
Argentina, Sudamericana                                20
                                                       ..
$$ - $$$, Panaderías y confiterías , Pizzería, Café     1
$, Francesa, Café, Delicatessen                         1
$$ - $$$, Café, Internacional, Bar de vinos             1
$$ - $$$, Café, Delicatessen, Gastropub                 1
Francesa, Estadounidense, Argentina, Sudamericana       1
Name: combinacion_tipos_comida, Length: 156, dtype: int64


In [94]:
frecuencia_combinaciones

Café                                                   81
$$ - $$$, Café                                         64
$$ - $$$, Café, Argentina, Sudamericana                45
                                                       30
Argentina, Sudamericana                                20
                                                       ..
$$ - $$$, Panaderías y confiterías , Pizzería, Café     1
$, Francesa, Café, Delicatessen                         1
$$ - $$$, Café, Internacional, Bar de vinos             1
$$ - $$$, Café, Delicatessen, Gastropub                 1
Francesa, Estadounidense, Argentina, Sudamericana       1
Name: combinacion_tipos_comida, Length: 156, dtype: int64

In [95]:
df_restaurants['postre'] = df_restaurants['type'].apply(lambda x:  1 if pd.notnull(x) and 'postre' in str(x).lower() else 0)
df_restaurants['delicatessen'] = df_restaurants['type'].apply(lambda x:  1 if pd.notnull(x) and 'delicatessen' in str(x).lower() else 0)
df_restaurants['bar'] = df_restaurants['type'].apply(lambda x: 1 if pd.notnull(x) and 'bar' in str(x).lower() else 0)
df_restaurants['panaderia'] = df_restaurants['type'].apply(lambda x: 1 if pd.notnull(x) and 'panadería' in str(x).lower() else 0)
df_restaurants['saludable'] = df_restaurants['type'].apply(lambda x:  1 if pd.notnull(x) and 'saludable' in str(x).lower() else 0)
df_restaurants['francesa'] = df_restaurants['combinacion_tipos_comida'].apply(lambda x: 1 if pd.notnull(x) and 'francesa' in str(x).lower() else 0)

In [96]:
df_restaurants['postre'].value_counts()

0    481
1     49
Name: postre, dtype: int64

In [97]:
df_restaurants['delicatessen'].value_counts()

0    503
1     27
Name: delicatessen, dtype: int64

In [98]:
df_restaurants['bar'].value_counts()

0    507
1     23
Name: bar, dtype: int64

In [99]:
df_restaurants['panaderia'].value_counts()

0    507
1     23
Name: panaderia, dtype: int64

In [100]:
df_restaurants['saludable'].value_counts()

0    518
1     12
Name: saludable, dtype: int64

In [101]:
df_restaurants['francesa'].value_counts()

0    503
1     27
Name: francesa, dtype: int64

In [102]:
df_restaurants['display_hours'].value_counts()

[{'days': 'dom - sáb', 'times': ['08:00 - 20:00']}]                                                       9
[{'days': 'mar - dom', 'times': ['09:00 - 20:00']}]                                                       4
[{'days': 'dom - sáb', 'times': ['00:00 - 23:59']}]                                                       3
[{'days': 'dom - sáb', 'times': ['07:00 - 23:00']}]                                                       3
[{'days': 'lun - vie', 'times': ['08:00 - 20:30']}, {'days': 'sáb - dom', 'times': ['09:00 - 20:30']}]    3
                                                                                                         ..
[{'days': 'lun - vie', 'times': ['07:00 - 21:00']}, {'days': 'sáb', 'times': ['09:00 - 18:00']}]          1
[{'days': 'dom - sáb', 'times': ['09:00 - 22:00']}]                                                       1
[{'days': 'lun - vie', 'times': ['08:30 - 19:30']}]                                                       1
[{'days': 'lun - vie', 'time

In [103]:
def extract_hours(display_hours):
    try:
        if isinstance(display_hours, float):
            return "Horario no disponible"

        hours_data = json.loads(str(display_hours).replace("'", "\""))
        opening_hours = []

        for day_info in hours_data:
            days = day_info.get('days', '')
            times = ', '.join(day_info.get('times', []))
            opening_hours.append(f"{days}: {times}")

        return ', '.join(opening_hours)
    except (json.JSONDecodeError, KeyError):
        return "Error al procesar el horario"


In [104]:
df_restaurants['extracted_hours'] = df_restaurants['display_hours'].apply(extract_hours)


In [105]:
df_restaurants['extracted_hours'].value_counts()

Horario no disponible                           328
dom - sáb: 08:00 - 20:00                          9
mar - dom: 09:00 - 20:00                          4
dom - sáb: 00:00 - 23:59                          3
dom - sáb: 07:00 - 23:00                          3
                                               ... 
lun - vie: 07:00 - 21:00, sáb: 09:00 - 18:00      1
dom - sáb: 09:00 - 22:00                          1
lun - vie: 08:30 - 19:30                          1
lun - vie: 08:00 - 21:00, sáb: 09:00 - 14:00      1
dom: 09:00 - 21:00, lun - sáb: 08:00 - 21:00      1
Name: extracted_hours, Length: 154, dtype: int64

In [106]:
def classify_time_category(opening_hours):
    # Busca patrones de horarios en el texto
    time_patterns = re.findall(r'\d{2}:\d{2} - \d{2}:\d{2}', opening_hours)

    if not time_patterns:
        return 'Otro'  # No se encontraron patrones de horarios

    # Tomamos solo el primer horario para simplificar
    first_time_pattern = time_patterns[0]

    if '08:00' <= first_time_pattern < '11:00':
        return 'Desayuno'
    elif '12:00' <= first_time_pattern < '14:00':
        return 'Almuerzo'
    elif '16:00' <= first_time_pattern < '18:00':
        return 'Merienda'
    elif '19:00' <= first_time_pattern < '22:00':
        return 'Cena'
    else:
        return 'Otro'

In [107]:
df_restaurants['time_category'] = df_restaurants['extracted_hours'].apply(classify_time_category)


In [108]:
df_restaurants['time_category'].value_counts()

Otro        372
Desayuno    148
Almuerzo      5
Merienda      5
Name: time_category, dtype: int64

In [109]:
df_restaurants[['display_hours','extracted_hours','time_category']]

,display_hours,extracted_hours,time_category
restaurant_id,,,
10048394,"[{'days': 'lun - vie', 'times': ['08:30 - 20:0...","lun - vie: 08:30 - 20:00, sáb - dom: 09:00 - 2...",Desayuno
2424768,"[{'days': 'dom', 'times': ['11:00 - 18:00']}, ...","dom: 11:00 - 18:00, lun: 10:00 - 18:00, mar - ...",Otro
1902671,NaN,Horario no disponible,Otro
3676783,"[{'days': 'dom', 'times': ['15:00 - 23:00']}, ...","dom: 15:00 - 23:00, lun: 09:00 - 22:00, mar: 0...",Otro
6563785,"[{'days': 'dom - sáb', 'times': ['08:00 - 01:0...",dom - sáb: 08:00 - 01:00,Desayuno
...,...,...,...
26768472,NaN,Horario no disponible,Otro
25762339,NaN,Horario no disponible,Otro
25330579,NaN,Horario no disponible,Otro


In [110]:
df_restaurants.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 530 entries, 10048394 to 25320045
Data columns (total 42 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   name                          530 non-null    object 
 1   num_reviews                   530 non-null    int64  
 2   latitude                      529 non-null    float64
 3   longitude                     529 non-null    float64
 4   timezone                      530 non-null    object 
 5   location                      530 non-null    object 
 6   raw_ranking                   467 non-null    float64
 7   ranking_position              467 non-null    float64
 8   ranking_caba                  467 non-null    float64
 9   rating                        467 non-null    float64
 10  price                         19 non-null     object 
 11  price_level                   332 non-null    object 
 12  phone                         379 non-null    object

guardar en CSV

In [111]:
df_restaurants.to_csv('restaurants_cafe_fixed.csv')

## Users

### Info

In [48]:
df_users.shape

(28391, 7)

In [49]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28391 entries, EE22C815B9937EC0FBB687C10B590769 to 2D47293DC564A0188DD073D4AC61B9AC
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   nick            28391 non-null  object
 1   user            28391 non-null  object
 2   locate          24198 non-null  object
 3   num_opinions    28391 non-null  int64 
 4   total_points    28391 non-null  int64 
 5   helpful_counts  28391 non-null  int64 
 6   critical_level  28391 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 1.7+ MB


###Eliminar duplicados

In [50]:
df_users = df_users[~df_users.index.duplicated(keep='first')]

In [51]:
df_users.shape

(19018, 7)

In [52]:
df_users.head(5)

,nick,user,locate,num_opinions,total_points,helpful_counts,critical_level
user_id,,,,,,,
EE22C815B9937EC0FBB687C10B590769,marcelamN9209FM,Marcela M,NaN,1,150,0,0
E4AB36841D4CDDA7A32AEF7A34C2C40C,Kathryn_Mzz,Kathryn M,"Santiago, Chile",60,0,13,5
665CA43E7EE0258B9E13E574141EDB21,charliecatalan,Chuck C,"Ciudad de Guatemala, Guatemala",19,0,66,3
F995C5EACA2B6E42E6441B4E8AAFD8F1,carolinapG3840LF,Carolina P,NaN,2,301,1,1
A1664FF313971E424002734D161E6339,negrar2017,Negra R,"Buenos Aires, Argentina",1,101,1,0


In [53]:
df_users['locate'].value_counts()

Buenos Aires, Argentina                 6236
Distrito Capital Federal, Argentina      456
Santiago, Chile                          338
Montevideo, Uruguay                      304
Provincia de Buenos Aires, Argentina     296
                                        ... 
ituzaingo                                  1
Las Cabras, Chile                          1
Montecarlo, Argentina                      1
santiago,chile                             1
Waldoboro, Maine                           1
Name: locate, Length: 2178, dtype: int64

In [54]:
df_users.to_csv('users_cafe_fixed.csv')

## Interactions

### Info

In [55]:
df_interactions.shape

(28390, 6)

In [56]:
df_interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28390 entries, 0 to 28389
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   restaurant_id  28390 non-null  object 
 1   user_id        28390 non-null  object 
 2   rating         28390 non-null  float64
 3   rating_date    28390 non-null  object 
 4   title          28388 non-null  object 
 5   opinion        28390 non-null  object 
dtypes: float64(1), object(5)
memory usage: 1.3+ MB


###Eliminar duplicados

In [57]:
df_interactions = df_interactions[~df_interactions.index.duplicated(keep='first')]

In [58]:
df_interactions.shape

(28390, 6)

In [59]:
df_interactions.head(5)

,restaurant_id,user_id,rating,rating_date,title,opinion
0,10048394,EE22C815B9937EC0FBB687C10B590769,5.0,7 de noviembre de 2023,Un lugar con mucho amor,"Hermoso lugar. Comida y café muy rico, con muc..."
1,10048394,E4AB36841D4CDDA7A32AEF7A34C2C40C,5.0,19 de octubre de 2023,Gran Cafe,La decoración del restaurante es muy bonita y ...
2,10048394,665CA43E7EE0258B9E13E574141EDB21,1.0,11 de octubre de 2023,Mala Atención al Cliente,Terrible experiencia\nMala atención al cliente...
3,10048394,665CA43E7EE0258B9E13E574141EDB21,1.0,10 de septiembre de 2023,T E R R I B L E,"Pase de nuevo por el local, porque tenia ganas..."
4,10048394,F995C5EACA2B6E42E6441B4E8AAFD8F1,5.0,19 de agosto de 2023,Imperdible,"Preciosa atención, café delicioso, y la comida..."


In [60]:
df_interactions.to_csv('interactions_cafe_fixed.csv')

### Formatear fechas

In [61]:
df_interactions['rating_date'] = pd.to_datetime(df_interactions['rating_date'], errors='coerce')


### Procesamieno de Texto

In [62]:
nltk.download('stopwords')
nltk.download('punkt')

# Suponiendo que las columnas 'opinion' y 'title' contienen texto
df_interactions['opinion'] = df_interactions['opinion'].astype(str)
df_interactions['title'] = df_interactions['title'].astype(str)

# Tokenización y eliminación de stopwords
stop_words = set(stopwords.words('spanish'))

def process_text(text):
    words = word_tokenize(text)
    filtered_words = [word.lower() for word in words if word.isalnum() and word.lower() not in stop_words]
    return ' '.join(filtered_words)

df_interactions['processed_opinion'] = df_interactions['opinion'].apply(process_text)
df_interactions['processed_title'] = df_interactions['title'].apply(process_text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [112]:
df_interactions.head(10)

,restaurant_id,user_id,rating,rating_date,title,opinion,processed_opinion,processed_title
0,10048394,EE22C815B9937EC0FBB687C10B590769,5.0,NaT,Un lugar con mucho amor,"Hermoso lugar. Comida y café muy rico, con muc...",hermoso lugar comida café rico amor trabajan s...,lugar amor
1,10048394,E4AB36841D4CDDA7A32AEF7A34C2C40C,5.0,NaT,Gran Cafe,La decoración del restaurante es muy bonita y ...,decoración restaurante bonita instalaciones li...,gran cafe
2,10048394,665CA43E7EE0258B9E13E574141EDB21,1.0,NaT,Mala Atención al Cliente,Terrible experiencia\nMala atención al cliente...,terrible experiencia mala atención cliente fat...,mala atención cliente
3,10048394,665CA43E7EE0258B9E13E574141EDB21,1.0,NaT,T E R R I B L E,"Pase de nuevo por el local, porque tenia ganas...",pase nuevo local tenia ganas cafe scone dulce ...,t r r i b l
4,10048394,F995C5EACA2B6E42E6441B4E8AAFD8F1,5.0,NaT,Imperdible,"Preciosa atención, café delicioso, y la comida...",preciosa atención café delicioso comida igual ...,imperdible
5,10048394,A1664FF313971E424002734D161E6339,5.0,NaT,Empezar el día acá!,"El café más lindo de Buenos Aires! La comida, ...",café lindo buenos aires comida cafe atención a...,empezar día acá
6,10048394,27D578CEC2D5C62D155240BF26CDEA34,5.0,NaT,Gran cafe,"Pequeña cafetería acogedora, excelente café y ...",pequeña cafetería acogedora excelente café del...,gran cafe
7,10048394,BE9919C45345B39401941F588AB5D3AB,5.0,NaT,Tenes que visitarlo,"Excelente atención, cordial y rápida. Adema de...",excelente atención cordial rápida adema café d...,tenes visitarlo
8,10048394,7278FE94AC529C98EA61DE21FB773395,5.0,NaT,Descubrimiento sorprendente,"Lindo ambiente, rápida atención, excelente caf...",lindo ambiente rápida atención excelente café ...,descubrimiento sorprendente
9,10048394,EFC10396A97959248C01B08889BE9B6A,5.0,NaT,Excelente café,"Excelente lugar para tomar un buen café, muy ...",excelente lugar tomar buen café aromático bien...,excelente café


In [123]:
df_interactions['restaurant_id'] = pd.to_numeric(df_interactions['restaurant_id'], errors='coerce')
df_interactions = df_interactions.dropna(subset=['restaurant_id']).astype({'restaurant_id': int})

In [113]:
df_restaurants = df_restaurants.rename(columns={"rating": "rating_rest"})

In [114]:
df_restaurants = df_restaurants.reset_index()

In [115]:
df_restaurants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 530 entries, 0 to 529
Data columns (total 43 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   restaurant_id                 530 non-null    int64  
 1   name                          530 non-null    object 
 2   num_reviews                   530 non-null    int64  
 3   latitude                      529 non-null    float64
 4   longitude                     529 non-null    float64
 5   timezone                      530 non-null    object 
 6   location                      530 non-null    object 
 7   raw_ranking                   467 non-null    float64
 8   ranking_position              467 non-null    float64
 9   ranking_caba                  467 non-null    float64
 10  rating_rest                   467 non-null    float64
 11  price                         19 non-null     object 
 12  price_level                   332 non-null    object 
 13  phone

In [124]:
df_interactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27285 entries, 0 to 28389
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   restaurant_id      27285 non-null  int64         
 1   user_id            27285 non-null  object        
 2   rating             27285 non-null  float64       
 3   rating_date        0 non-null      datetime64[ns]
 4   title              27285 non-null  object        
 5   opinion            27285 non-null  object        
 6   processed_opinion  27285 non-null  object        
 7   processed_title    27285 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 1.9+ MB


In [116]:
perf_restaurants = df_restaurants[["restaurant_id","rating_rest","califications_comida","califications_servicio","califications_calidad_precio","califications_ambiente"]]

In [117]:
user_id='665CA43E7EE0258B9E13E574141EDB21'

In [125]:
perf_usuario = df_interactions[(df_interactions["user_id"] == user_id) & (df_interactions["rating"] > 0)].merge(perf_restaurants, on="restaurant_id")

In [126]:
perf_usuario

,restaurant_id,user_id,rating,rating_date,title,opinion,processed_opinion,processed_title,rating_rest,califications_comida,califications_servicio,califications_calidad_precio,califications_ambiente
0,10048394,665CA43E7EE0258B9E13E574141EDB21,1.0,NaT,Mala Atención al Cliente,Terrible experiencia\nMala atención al cliente...,terrible experiencia mala atención cliente fat...,mala atención cliente,5.0,4.5,4.5,4.5,3.5
1,10048394,665CA43E7EE0258B9E13E574141EDB21,1.0,NaT,T E R R I B L E,"Pase de nuevo por el local, porque tenia ganas...",pase nuevo local tenia ganas cafe scone dulce ...,t r r i b l,5.0,4.5,4.5,4.5,3.5


In [127]:
for c in perf_usuario.columns:
  if c.startswith("califications_"):
    perf_usuario[c] = perf_usuario[c] * perf_usuario["rating"]


In [128]:
perf_usuario

,restaurant_id,user_id,rating,rating_date,title,opinion,processed_opinion,processed_title,rating_rest,califications_comida,califications_servicio,califications_calidad_precio,califications_ambiente
0,10048394,665CA43E7EE0258B9E13E574141EDB21,1.0,NaT,Mala Atención al Cliente,Terrible experiencia\nMala atención al cliente...,terrible experiencia mala atención cliente fat...,mala atención cliente,5.0,4.5,4.5,4.5,3.5
1,10048394,665CA43E7EE0258B9E13E574141EDB21,1.0,NaT,T E R R I B L E,"Pase de nuevo por el local, porque tenia ganas...",pase nuevo local tenia ganas cafe scone dulce ...,t r r i b l,5.0,4.5,4.5,4.5,3.5


In [129]:
perf_usuario = perf_usuario.drop(columns=["restaurant_id", "rating"]).groupby("user_id").mean()

<ipython-input-129-721c4a30393a>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  perf_usuario = perf_usuario.drop(columns=["restaurant_id", "rating"]).groupby("user_id").mean()


In [130]:
perf_usuario

,rating_rest,califications_comida,califications_servicio,califications_calidad_precio,califications_ambiente
user_id,,,,,
665CA43E7EE0258B9E13E574141EDB21,5.0,4.5,4.5,4.5,3.5


In [131]:
perf_usuario = perf_usuario / perf_usuario.sum(axis=1)[0] # normalizo

In [132]:
perf_usuario


,rating_rest,califications_comida,califications_servicio,califications_calidad_precio,califications_ambiente
user_id,,,,,
665CA43E7EE0258B9E13E574141EDB21,0.227273,0.204545,0.204545,0.204545,0.159091


In [138]:
for g in perf_usuario.columns:
    perf_restaurants[g] = perf_restaurants[g] * perf_usuario[g][0]

<ipython-input-138-c089e0f0fd69>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perf_restaurants[g] = perf_restaurants[g] * perf_usuario[g][0]


In [139]:
perf_usuario.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 665CA43E7EE0258B9E13E574141EDB21 to 665CA43E7EE0258B9E13E574141EDB21
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   rating_rest                   1 non-null      float64
 1   califications_comida          1 non-null      float64
 2   califications_servicio        1 non-null      float64
 3   califications_calidad_precio  1 non-null      float64
 4   califications_ambiente        1 non-null      float64
dtypes: float64(5)
memory usage: 156.0+ bytes


In [136]:
perf_restaurants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 530 entries, 0 to 529
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   restaurant_id                 530 non-null    int64  
 1   rating_rest                   467 non-null    float64
 2   califications_comida          255 non-null    float64
 3   califications_servicio        255 non-null    float64
 4   califications_calidad_precio  255 non-null    float64
 5   califications_ambiente        73 non-null     float64
dtypes: float64(5), int64(1)
memory usage: 25.0 KB


In [140]:
restaurantes_vistos_o_puntuados = df_interactions.loc[df_interactions["user_id"] == user_id, "restaurant_id"].tolist()

In [141]:
restaurantes_vistos_o_puntuados

[10048394, 10048394]

In [142]:

recomendaciones = [restaurant for restaurant in df_restaurants.sum(axis=1).sort_values(ascending=False).index
  if restaurant not in restaurantes_vistos_o_puntuados][:9]

<ipython-input-142-f1a076fb07a7>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  recomendaciones = [restaurant for restaurant in df_restaurants.sum(axis=1).sort_values(ascending=False).index


In [143]:
recomendaciones

[327, 495, 490, 502, 506, 496, 525, 488, 467]

# Merged dataset

In [ ]:
df_restaurants.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 530 entries, 10048394 to 25320045
Data columns (total 42 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   name                          530 non-null    object 
 1   num_reviews                   530 non-null    int64  
 2   latitude                      529 non-null    float64
 3   longitude                     529 non-null    float64
 4   timezone                      530 non-null    object 
 5   location                      530 non-null    object 
 6   raw_ranking                   467 non-null    float64
 7   ranking_position              467 non-null    float64
 8   ranking_caba                  467 non-null    float64
 9   rating                        467 non-null    float64
 10  price                         19 non-null     object 
 11  price_level                   332 non-null    object 
 12  phone                         379 non-null    object

In [ ]:
df_interactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28390 entries, 0 to 28389
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   restaurant_id      28390 non-null  object        
 1   user_id            28390 non-null  object        
 2   rating             28390 non-null  float64       
 3   rating_date        0 non-null      datetime64[ns]
 4   title              28390 non-null  object        
 5   opinion            28390 non-null  object        
 6   processed_opinion  28390 non-null  object        
 7   processed_title    28390 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 1.9+ MB


In [ ]:
merged_df = pd.merge(df_interactions, df_restaurants, on='restaurant_id')
merged_df = pd.merge(merged_df, df_users, on='user_id')

In [ ]:
merged_df

,restaurant_id,user_id,rating_x,rating_date,title,opinion,processed_opinion,processed_title,name,num_reviews,...,francesa,extracted_hours,time_category,nick,user,locate,num_opinions,total_points,helpful_counts,critical_level
0,10048394.0,EE22C815B9937EC0FBB687C10B590769,5.0,NaT,Un lugar con mucho amor,"Hermoso lugar. Comida y café muy rico, con muc...",hermoso lugar comida café rico amor trabajan s...,lugar amor,Vive Cafe,357,...,0,"lun - vie: 08:30 - 20:00, sáb - dom: 09:00 - 2...",Desayuno,marcelamN9209FM,Marcela M,NaN,1,150,0,0
1,10048394.0,E4AB36841D4CDDA7A32AEF7A34C2C40C,5.0,NaT,Gran Cafe,La decoración del restaurante es muy bonita y ...,decoración restaurante bonita instalaciones li...,gran cafe,Vive Cafe,357,...,0,"lun - vie: 08:30 - 20:00, sáb - dom: 09:00 - 2...",Desayuno,Kathryn_Mzz,Kathryn M,"Santiago, Chile",60,0,13,5
2,7619423.0,E4AB36841D4CDDA7A32AEF7A34C2C40C,5.0,NaT,Que bien se come!,La decoración de la cafetería es muy bonita y ...,decoración cafetería bonita instalaciones limp...,bien come,Dulce Charlotte Tienda,98,...,1,dom - sáb: 09:00 - 20:00,Desayuno,Kathryn_Mzz,Kathryn M,"Santiago, Chile",60,0,13,5
3,10048394.0,665CA43E7EE0258B9E13E574141EDB21,1.0,NaT,Mala Atención al Cliente,Terrible experiencia\nMala atención al cliente...,terrible experiencia mala atención cliente fat...,mala atención cliente,Vive Cafe,357,...,0,"lun - vie: 08:30 - 20:00, sáb - dom: 09:00 - 2...",Desayuno,charliecatalan,Chuck C,"Ciudad de Guatemala, Guatemala",19,0,66,3
4,10048394.0,665CA43E7EE0258B9E13E574141EDB21,1.0,NaT,T E R R I B L E,"Pase de nuevo por el local, porque tenia ganas...",pase nuevo local tenia ganas cafe scone dulce ...,t r r i b l,Vive Cafe,357,...,0,"lun - vie: 08:30 - 20:00, sáb - dom: 09:00 - 2...",Desayuno,charliecatalan,Chuck C,"Ciudad de Guatemala, Guatemala",19,0,66,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27279,8055047.0,E3DFB93E530C0FF35B0BBA186B5CBF5F,1.0,NaT,Nueva gerencia destruyó todo,Ahora es carísimo y redujeron notablemente las...,ahora carísimo redujeron notablemente porcione...,nueva gerencia destruyó,Stefano,29,...,0,Horario no disponible,Otro,rafaeltF3061VQ,Rafael T,NaN,1,100,0,0
27280,8055047.0,723FC09F369F8398748891688327F98D,1.0,NaT,Almuerzo lamentable,"La atención es pésima, no hay nadie que contro...",atención pésima nadie controle nadie importe a...,almuerzo lamentable,Stefano,29,...,0,Horario no disponible,Otro,federichi65,Federica M,NaN,17,0,3,3
27281,8055047.0,5F615CCB3790330DE4959DBE2D0D4084,2.0,NaT,Decepcion,Decididamente no lo recomendaría. Pedí un bro...,decididamente recomendaría pedí brownie carta ...,decepcion,Stefano,29,...,0,Horario no disponible,Otro,Guille25,Guille25,"Buenos Aires, Argentina",37,0,18,5
27282,8055047.0,BA1916F751B5F6E14C1FCC60BF729340,1.0,NaT,"espantoso, la atencion horrible","Fuimos a merendar con mi hija, pedimos la cart...",merendar hija pedimos carta elejimos torta 3 p...,espantoso atencion horrible,Stefano,29,...,0,Horario no disponible,Otro,Ana_laplata2014,Ana Maria E,"La Plata, Argentina",69,0,40,5


In [ ]:
# Asegurémonos de que las columnas necesarias estén presentes
required_columns = ['user_id', 'restaurant_id', 'rating_x']
if not set(required_columns).issubset(merged_df.columns):
    raise ValueError(f"Las columnas requeridas ({required_columns}) no están presentes en el DataFrame.")

# Crear un objeto Reader con el rango de calificaciones
reader = Reader(rating_scale=(1, 5))

# Cargar el conjunto de datos desde el DataFrame
data = Dataset.load_from_df(merged_df[required_columns], reader)

# Dividir el conjunto de datos en entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Configurar y entrenar el modelo KNN Basic
sim_options = {'name': 'cosine', 'user_based': True}
model = KNNBasic(sim_options=sim_options)
model.fit(trainset)

# Predecir las calificaciones para el conjunto de prueba
predictions = model.test(testset)

# Evaluar el rendimiento del modelo
accuracy.rmse(predictions)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.1044


1.1044196631064744